In [ ]:
import sys
sys.path.append('/nfs/home/genovese/thesis-wildfire-genovese/only_human_study/')
import utils
from importlib import reload
reload(utils)
from utils import *

In [ ]:
scores = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/only_human_study/outputs/sample_a/regression_rf_rfe_tracking.csv',  index_col=0)

In [ ]:
X_train_val = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/only_human_study/database/sample_a/X_train_val.csv')
X_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/only_human_study/database/sample_a/X_test.csv')
y_train_val = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/only_human_study/database/sample_a/y_train_val.csv')
y_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/only_human_study/database/sample_a/y_test.csv')

In [7]:
final_train = X_train_val.loc[:, scores.index].loc[:, ~scores['iteration_26'].isna()]
final_test = X_test.loc[:, scores.index].loc[:, ~scores['iteration_26'].isna()]

In [ ]:
final_train.to_csv('/nfs/home/genovese/thesis-wildfire-genovese/only_human_study/database/sample_a/final_X_train_val.csv', index=False)
final_test.to_csv('/nfs/home/genovese/thesis-wildfire-genovese/only_human_study/database/sample_a/final_X_test.csv', index=False)

---